In [1]:
import torch
from time import time

## Anderson-Darling

In [17]:
def anderson_darling(X_obs, X_gen):
    n_test, n_sta = X_gen.size()
    
    X_gen = X_gen.sort(axis=0).values
    u = torch.zeros_like(X_gen)
    
    for i in range(n_test):
        for s in range(n_sta):
            u[i, s] = torch.sum(X_obs[:, s] <= X_gen[i, s]) + 1
    u /= n_test + 2
    
    W = torch.zeros(n_sta)
    for s in range(n_sta):
        W[s] = sum([(2*i-1)*(torch.log(u[i, s]) + torch.log(1-u[-(i+1), s])) for i in range(n_test)])
        W[s] /= -n_test
        W[s] -= n_test
    
    return torch.mean(W)

## Absolute Kendall

In [18]:
def Ri(X, i):
    n_test = X.size(0)
    X_ref = X[i, :].repeat((n_test, 1))  # copy the i-th observation and repeat n_test times
    return torch.sum(torch.prod(X - X_ref < 0, axis=1))/(n_test-1)  # prod of bool

In [19]:
def kendall(X_obs, X_gen):
    n_test = X_obs.size(0)
    R = torch.tensor([Ri(X_obs, i) for i in range(n_test)]).sort().values  # sort to get the order statistics
    R_tilde = torch.tensor([Ri(X_gen, i) for i in range(n_test)]).sort().values
    return torch.mean(torch.abs(R - R_tilde))

## Test

In [22]:
obs = torch.randn((1000, 6))
gen = torch.randn_like(obs)

In [23]:
anderson_darling(obs, gen)

tensor(-2.3403)

In [24]:
kendall(obs, gen)

tensor(0.0017)

In [25]:
obs = torch.randn((10000, 6))
gen = torch.randn((10000, 6))

In [26]:
t0 = time()
anderson_darling(obs, gen)
t1 = time()
kendall(obs, gen)
t2 = time()

In [27]:
print(t2-t1)
print(t1-t0)

4.3854992389678955
4.662496328353882
